<a href="https://colab.research.google.com/github/xinformatics/streamlit_workshop/blob/main/whisper_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Libraries**

In [1]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install streamlit
!npm install localtunnel
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-gjozx37j
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-gjozx37j
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=29d367aaaf03eeb74d3440123c4fe6a47fab5b8270282d2f11145ca2d268df09
  Stored in directory: /tmp/pip-ephem-wheel-cache-w9ntaxze/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

**Install ffmpeg**

In [1]:
# !sudo apt update && sudo apt install ffmpeg

In [2]:
# !pip install streamlit

In [3]:
# !npm install localtunnel

In [2]:
# !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
# import subprocess
# subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

nohup: appending output to 'nohup.out'


In [3]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://spray-actions-unlock-discipline.trycloudflare.com


In [13]:
%%writefile app.py
import streamlit as st
import subprocess
import os

# Title of the application
st.title('Streamlit Speech to text Converter')

# File Uploader
uploaded_file = st.file_uploader("Choose an MP3 file", type="mp3")

# Process Button
if uploaded_file is not None:
    # Save the uploaded file
    with open("temp.mp3", "wb") as f:
        f.write(uploaded_file.getvalue())

    # Button to start processing
    if st.button('Process'):
        # Run the Whisper command
        command = f"whisper temp.mp3 --model medium --language English --output_format txt"
        process = subprocess.run(command, shell=True, capture_output=True, text=True)

        if process.returncode == 0:
            # Assuming the transcription is printed to stdout
            transcription = process.stdout
            st.text_area("Transcription", transcription, height=300)
        else:
            st.error("Error in transcription")

# Cleanup: Remove the temporary file if it exists
if os.path.exists("temp.mp3"):
    os.remove("temp.mp3")

Overwriting app.py


In [9]:
%%writefile app.py
import streamlit as st
import subprocess
import os

# Title of the application
st.title('Streamlit Speech to text Converter')

# File Uploader
uploaded_file = st.file_uploader("Choose an MP3 file", type="mp3")

# Process Button
if uploaded_file is not None:
    # Save the uploaded file
    with open("temp.mp3", "wb") as f:
        f.write(uploaded_file.getvalue())

    # Button to start processing
    if st.button('Process'):
        # Run the Whisper command
        command = f"whisper temp.mp3 --model medium --output temp.txt"
        process = subprocess.run(command, shell=True, capture_output=True, text=True)
        print('after whisper')
        # Check if the process was successful
        if process.returncode == 0:
            # Assuming the transcription is printed to stdout
            # transcription = process.stdout
            # st.text_area("Transcription", transcription, height=300)
            if os.path.exists("temp.txt"):
                with open("temp.txt", "r") as file:
                    transcription = file.read()
                st.text_area("Transcription", transcription, height=300)
                os.remove("temp.txt")  # Remove the temp.txt file after displaying its contents
            else:
                st.error("Transcription file not found.")
        else:
            st.error("Error in transcription")

# Cleanup: Remove the temporary audio file if it exists
if os.path.exists("temp.mp3"):
    os.remove("temp.mp3")

Overwriting app.py


In [15]:
!streamlit run /content/app.py &>/content/logs.txt &

In [16]:
#### check whisper
# !whisper "temp.mp3" --model medium